In [1]:
from itertools import combinations
import networkx as nx
import matplotlib.pyplot as plt
import random
import math
from copy import deepcopy

In [2]:
#BRUTE_FORCE
def is_graph_minimum_vertex_cover(G, vertex_cover):
    for u in G:
        for v in G[u]:
            if u not in vertex_cover and v not in vertex_cover:
                return False
    return True
    
def brute_force(G):
    vertices = list(G.nodes())
    n = len(vertices)
    
    for r in range(1, n+1):
        for subset in combinations(vertices, r):
            vertex_cover = set(subset)
            if is_graph_minimum_vertex_cover(G, vertex_cover):
                return vertex_cover, len(vertex_cover)
    
    return None, None

In [3]:
#S-METAHEURSITIC
#------------------------------------------------------
#SHARED
def initial_solution(graph):
    num_nodes = len(graph.nodes())
    solution_list = [False] * num_nodes
    
    while not is_valid_cover(solution_list, graph):
        remaining_nodes = [i for i in range(num_nodes) if not solution_list[i]]
        random_node = random.choice(remaining_nodes)
        solution_list[random_node] = True
    
    return solution_list

def calculate_cost(vertex_cover, graph):
    return sum(vertex_cover)  

def is_valid_cover(vertex_cover, graph):
    for u, v in graph.edges():
        if not vertex_cover[u] and not vertex_cover[v]:
            return False
    return True


#------------------------------------------------------
#SIMMULATED_ANNEALING

def get_neighbor(vertex_cover, graph):
    num_nodes = len(vertex_cover)
    new_solution = deepcopy(vertex_cover)
    random_idx = random.randrange(len(vertex_cover))
    
    new_solution[random_idx] = not new_solution[random_idx]
    
    neighbor = new_solution
    
    if not is_valid_cover(neighbor, graph):

        uncovered_edges = [(u, v) for u, v in graph.edges() if not neighbor[u] and not neighbor[v]]
        while uncovered_edges:
            u, v = random.choice(uncovered_edges)
            if not neighbor[u]:
                neighbor[u] = True
            if not neighbor[v]:
                neighbor[v] = True
            uncovered_edges = [(u, v) for u, v in graph.edges() if not neighbor[u] and not neighbor[v]]
    
    return neighbor

def simulated_annealing(graph, max_iter, k):
    current_solution = initial_solution(graph)
    current_cost = calculate_cost(current_solution, graph)
    best_solution = current_solution
    best_cost = current_cost
    cost_progress = [best_cost]
    
    for i in range(1, max_iter):
        neighbor = get_neighbor(current_solution, graph)
        neighbor_cost = calculate_cost(neighbor, graph)
        cost_progress.append(neighbor_cost)
        
        if neighbor_cost < current_cost:
            current_cost = neighbor_cost
            current_solution = neighbor 
            if neighbor_cost < best_cost:
                best_solution = neighbor
                best_cost = neighbor_cost
        else:
            p = 1 / i ** k
            q = random.random()
            if q < p:
                current_cost = neighbor_cost
                current_solution = neighbor
                

    return best_solution, best_cost, cost_progress


#------------------------------------------------------
#VNS
def shaking(solution, k, graph):
    num_nodes = len(solution)
    
    k = min(k, num_nodes)

    chosen_indices = random.sample(range(num_nodes), k)
    
    new_solution = solution.copy()

    for idx in chosen_indices:
        new_solution[idx] = not new_solution[idx]
    
    if not is_valid_cover(new_solution, graph):

        uncovered_edges = [(u, v) for u, v in graph.edges() if not new_solution[u] and not new_solution[v]]
        
        while uncovered_edges:
            u, v = random.choice(uncovered_edges)
            if not new_solution[u]:
                new_solution[u] = True
            if not new_solution[v]:
                new_solution[v] = True
                
            uncovered_edges = [(u, v) for u, v in graph.edges() if not new_solution[u] and not new_solution[v]]
    
    return new_solution

def local_search_best_improvement(solution, graph):
    improved = True
    
    while improved:
        improved = False
        best_solution = solution.copy()
        best_value = calculate_cost(best_solution, graph)
        
        for node in range(len(solution)):
            if best_solution[node]:  
                temp_solution = best_solution.copy()
                temp_solution[node] = False
                if is_valid_cover(temp_solution, graph):
                    temp_value = calculate_cost(temp_solution, graph)
                    if temp_value < best_value:
                        best_value = temp_value
                        best_solution = temp_solution
                        improved = True
                    elif random.random() < 0.1: 
                        best_solution = temp_solution
                        best_value = temp_value
                        improved = True
        
        uncovered_edges = [(u, v) for u, v in graph.edges() if not best_solution[u] and not best_solution[v]]
        for u, v in uncovered_edges:
            temp_solution = best_solution.copy()
            if not temp_solution[u]:
                temp_solution[u] = True
            if not temp_solution[v]:
                temp_solution[v] = True
            temp_value = calculate_cost(temp_solution, graph)
            if temp_value < best_value:
                best_value = temp_value
                best_solution = temp_solution
                improved = True
        
        solution = best_solution
    
    return solution

def vns(graph, max_iters, k_max, move_prob):
    solution = initial_solution(graph)
    value = calculate_cost(solution, graph)
    cost_progress = [value]
    
    
    for i in range(1, max_iters):
        for k in range(1, k_max + 1):
            new_solution = shaking(solution, k, graph)
            new_solution = local_search_best_improvement(new_solution, graph)
            new_value = calculate_cost(new_solution, graph)
            if new_value < value or (new_value == value and random.random() < move_prob):
                value = new_value
                solution = new_solution
                break
        cost_progress.append(new_value)

    return solution, value, cost_progress
